# load env

In [1]:
from datetime import date, datetime
from typing import List
import json

from apikey import apikey, API_KEY, BASE_URL, DEPLOYMENT_NAME
from langchain.chat_models import AzureChatOpenAI
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
from langchain.prompts import ChatPromptTemplate

import os
os.environ['OPENAI_API_KEY'] = apikey

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings("ignore")


In [2]:
from langchain.agents import load_tools, initialize_agent
from langchain.agents import tool
from langchain.agents import AgentType


# Model

In [3]:
llm_azure = AzureChatOpenAI(
    openai_api_base=BASE_URL,
    openai_api_version="2023-05-15",
    deployment_name=DEPLOYMENT_NAME,
    openai_api_key=API_KEY,
    openai_api_type = "azure",
    temperature=0
)

In [4]:
# test
llm_azure([HumanMessage(content="Translate this sentence from English to Chinese. I love programming.")])

AIMessage(content='我喜欢编程。', additional_kwargs={}, example=False)

In [5]:
# llm = ChatOpenAI(temperature=0)


In [6]:
tools = load_tools(["llm-math", "wikipedia"], llm=llm_azure)


# Agent

In [7]:
@tool
def time(text: str) ->str:
    """
    Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date mathmatics should occur \
    outside this function.
    """
    return str(date.today())

In [12]:
@tool
def words_segm(text: str, llm) ->List[str]:
    """
    Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date mathmatics should occur \
    outside this function.
    """
    
    return text, llm


In [14]:
@tool
def words_segm(text: str, llm) ->List[str]:
    """
    Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date mathmatics should occur \
    outside this function.
    """
    template_string = """
    Please extract the key participle of the following text,\
    and only numeric, date, and category type participles are required, \
    then return the results as a Python list.
    text: ```{text}```"""
    
    prompt_template = ChatPromptTemplate.from_template(template_string)
    messages = prompt_template.format_messages(text = text)
    segment_list = eval(llm(messages).content)

    return segment_list


In [6]:
agent = initialize_agent(
    tools + [time],
    llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose=True
)

In [38]:
try:
    result = agent("whats the date today?")
except: 
    print("exception on external access")

exception on external access


In [7]:
agent("whats the date today?")



> Entering new AgentExecutor chain...
Question: What's the date today?
Thought: I can use the `time` tool to get the current date.
Action:
```
{
  "action": "time",
  "action_input": ""
}
```

Observation: 2023-07-20
Thought:Could not parse LLM output: I now know the final answer.
Observation: Invalid or incomplete response
Thought:I apologize for the incomplete response. Let me provide you with the correct information.

Question: What's the date today?
Thought: I can use the `time` tool to get the current date.
Action:
```
{
  "action": "time",
  "action_input": ""
}
```


Observation: 2023-07-20
Thought:I now know the final answer.
Final Answer: The date today is July 20, 2023.

> Finished chain.


{'input': 'whats the date today?',
 'output': 'The date today is July 20, 2023.'}